In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
import re

In [25]:
depressed_dataset = pd.read_csv("D:\Workspace\OVGU\SM_Depression\src\depTweetsOrderedByDateTimeWindowURLTokens.csv")

In [26]:
depressed_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61972 entries, 0 to 61971
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   accountname       61972 non-null  object 
 1   description       61483 non-null  object 
 2   favorite_count    61937 non-null  float64
 3   location          51551 non-null  object 
 4   masked_user_id    61972 non-null  object 
 5   retweet_count     61968 non-null  object 
 6   timestamp         61972 non-null  object 
 7   tweets            61972 non-null  object 
 8   twittername       61972 non-null  object 
 9   user_type         61972 non-null  float64
 10  format            61972 non-null  int64  
 11  new_date          61972 non-null  object 
 12  time_window       61972 non-null  int64  
 13  tweets_url_token  61972 non-null  object 
 14  cleanData         61728 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 7.1+ MB


In [27]:
print(tf.__version__)

2.2.0


In [28]:
depressed_dataset.head()

,accountname,description,favorite_count,location,masked_user_id,retweet_count,timestamp,tweets,twittername,user_type,format,new_date,time_window,tweets_url_token,cleanData
0,Cmdr. Nasty Tinkerbell of the 4077th,NSFW \r\r\r\r\nWoman \r\r\r\r\nDisabled \r\r\r...,0.0,Exploring the universe,depressed_user_11,0.0,24-06-2020 00:00,@Twitter it's about time.\r\r\r\r\n\r\r\r\r\nL...,CmdrTinkerBell,1.0,1,2020-06-24 00:00,1,@Twitter it's about time.\r\r\r\r\n\r\r\r\r\nL...,it's about time. Lol IT'S NOT FAIR [URL] [URL]
1,Cmdr. Nasty Tinkerbell of the 4077th,NSFW \r\r\r\r\nWoman \r\r\r\r\nDisabled \r\r\r...,0.0,Exploring the universe,depressed_user_11,0.0,24-06-2020 00:02,"@w_terrence @PearlCathey8 Like deplorable, lik...",CmdrTinkerBell,1.0,1,2020-06-24 00:02,1,"@w_terrence @PearlCathey8 Like deplorable, lik...","Like deplorable, like donnie, have to use cont..."
2,Cmdr. Nasty Tinkerbell of the 4077th,NSFW \r\r\r\r\nWoman \r\r\r\r\nDisabled \r\r\r...,0.0,Exploring the universe,depressed_user_11,0.0,24-06-2020 00:05,@HarleyQuinnlif3 Dick always ruin things lol,CmdrTinkerBell,1.0,1,2020-06-24 00:05,1,@HarleyQuinnlif3 Dick always ruin things lol,Dick always ruin things lol
3,Cmdr. Nasty Tinkerbell of the 4077th,NSFW \r\r\r\r\nWoman \r\r\r\r\nDisabled \r\r\r...,2.0,Exploring the universe,depressed_user_11,0.0,24-06-2020 00:13,After seeing pics of todays rally in Phoenix I...,CmdrTinkerBell,1.0,1,2020-06-24 00:13,1,After seeing pics of todays rally in Phoenix I...,After seeing pics of todays rally in Phoenix I...
4,Cmdr. Nasty Tinkerbell of the 4077th,NSFW \r\r\r\r\nWoman \r\r\r\r\nDisabled \r\r\r...,2.0,Exploring the universe,depressed_user_11,0.0,24-06-2020 00:13,if I do get covid19 becuase I'm surrounded by ...,CmdrTinkerBell,1.0,1,2020-06-24 00:13,1,if I do get covid19 becuase I'm surrounded by ...,if I do get covid19 becuase I'm surrounded by ...


In [29]:
print(depressed_dataset.isnull().sum())


depressed_dataset = depressed_dataset.replace(np.nan, '', regex=True)

print(depressed_dataset.isnull().sum())
print("")
# print(depressed_dataset.info())

accountname             0
description           489
favorite_count         35
location            10421
masked_user_id          0
retweet_count           4
timestamp               0
tweets                  0
twittername             0
user_type               0
format                  0
new_date                0
time_window             0
tweets_url_token        0
cleanData             244
dtype: int64
accountname         0
description         0
favorite_count      0
location            0
masked_user_id      0
retweet_count       0
timestamp           0
tweets              0
twittername         0
user_type           0
format              0
new_date            0
time_window         0
tweets_url_token    0
cleanData           0
dtype: int64



In [30]:
tweet_text = depressed_dataset.loc[:,'cleanData']

In [31]:
tweet_text

0           it's about time. Lol IT'S NOT FAIR [URL] [URL]
1        Like deplorable, like donnie, have to use cont...
2                              Dick always ruin things lol
3        After seeing pics of todays rally in Phoenix I...
4        if I do get covid19 becuase I'm surrounded by ...
                               ...                        
61967    You need not to be afraid anymore. You are lov...
61968    I did register but I still didn't get the link...
61969               You have a beautiful name too! ð???â?�
61970                                                 yes!
61971                                 Can I get the link??
Name: cleanData, Length: 61972, dtype: object

In [32]:
def preprocessing(String): #default value is always true for stemming and stopwords
#     print(String)
#     print("")
    '''
    This function is used for preprocessing
    - Tokenization
    - Stemming
    - Stop Words
        
    '''
    
    influential_words = re.sub(r'[^a-zA-Z0-9_\[\]]', ' ', String)
#     print(influential_words)
#     tokens = nltk.word_tokenize(String)
#     print(tokens)
#     token = [word for word in tokens if word.isalpha()]
#     print(token)
#     influential_words = " ".join(token)
#     print(influential_words)
    influential_words = influential_words.lower()
#     print(influential_words)
#     influential_words = influential_words.split()
#     print(influential_words)
#     stemwords_string = " ".join(influential_words)
#     print(stemwords_string)
    return influential_words

In [33]:
final_text = []
for i in range(0,len(depressed_dataset.axes[0])):
    final_text.append(str(preprocessing(tweet_text[i])))

In [34]:
depressed_dataset['cleanData'] = final_text

In [35]:
del final_text,tweet_text,i

In [36]:
#clean special characters from cleanData row to apply BERT embeddings
def preprocess(tweets):
    tweets = tweets.replace("(<br/>)", "")
    tweets = tweets.replace('(<a).*(>).*(</a>)', '')
    tweets = tweets.replace('(&amp)', '')
    tweets = tweets.replace('(&gt)', '')
    tweets = tweets.replace('(&lt)', '')
    tweets = tweets.replace('(â€™)', '')
    tweets = tweets.replace('(\xa0)', ' ')
    tweets = tweets.replace('(https)', ' ')
    tweets = tweets.replace('(RT)', ' ')
    tweets = tweets.replace('(ðÿ)', '  ')
    tweets = tweets.replace('(\n)', '  ')
    return tweets

In [37]:
depressed_dataset['cleanData'] = depressed_dataset['cleanData'].apply(preprocess).astype(str)


In [38]:
import sys

In [39]:
depressed_dataset_1 = depressed_dataset[depressed_dataset.time_window == 1]
depressed_dataset_2 = depressed_dataset[depressed_dataset.time_window == 2]
depressed_dataset_3 = depressed_dataset[depressed_dataset.time_window == 3]
depressed_dataset_4 = depressed_dataset[depressed_dataset.time_window == 4]
depressed_dataset_5 = depressed_dataset[depressed_dataset.time_window == 5]
depressed_dataset_6 = depressed_dataset[depressed_dataset.time_window == 6]
depressed_dataset_7 = depressed_dataset[depressed_dataset.time_window == 7]
depressed_dataset_8 = depressed_dataset[depressed_dataset.time_window == 8]
depressed_dataset_9 = depressed_dataset[depressed_dataset.time_window == 9]
depressed_dataset_10 = depressed_dataset[depressed_dataset.time_window == 10]
depressed_dataset_11 = depressed_dataset[depressed_dataset.time_window == 11]
depressed_dataset_12 = depressed_dataset[depressed_dataset.time_window == 12]
depressed_dataset_13 = depressed_dataset[depressed_dataset.time_window == 13]
depressed_dataset_14 = depressed_dataset[depressed_dataset.time_window == 14]


for i in range(1,15):
    filename = "depressed_dataset_"+str(i)
    print(filename)
    print(getattr(sys.modules[__name__], f"depressed_dataset_{i}").shape)
    print("")

depressed_dataset_1
(700, 15)

depressed_dataset_2
(1093, 15)

depressed_dataset_3
(3223, 15)

depressed_dataset_4
(4868, 15)

depressed_dataset_5
(5463, 15)

depressed_dataset_6
(6878, 15)

depressed_dataset_7
(7786, 15)

depressed_dataset_8
(8278, 15)

depressed_dataset_9
(8394, 15)

depressed_dataset_10
(5696, 15)

depressed_dataset_11
(4408, 15)

depressed_dataset_12
(2729, 15)

depressed_dataset_13
(1569, 15)

depressed_dataset_14
(887, 15)



In [40]:
model_directory = "D:\Workspace\OVGU\SM_Depression\scripts\saved_models\kaggle_finetune_epoch10_es_new" # change this to your preferred location


In [41]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping
import json
import gc

In [42]:
gc.collect()

64

In [43]:
loaded_tokenizer = DistilBertTokenizer.from_pretrained(model_directory)
loaded_model = TFDistilBertForSequenceClassification.from_pretrained(model_directory,output_hidden_states=True)

Some layers from the model checkpoint at D:\Workspace\OVGU\SM_Depression\scripts\saved_models\kaggle_finetune_epoch10_es_new were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at D:\Workspace\OVGU\SM_Depression\scripts\saved_models\kaggle_finetune_epoch10_es_new and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stre

In [44]:
loaded_model.summary()

Model: "tf_distil_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  3845      
_________________________________________________________________
dropout_39 (Dropout)         multiple                  0         
Total params: 66,957,317
Trainable params: 66,957,317
Non-trainable params: 0
_________________________________________________________________


In [162]:
# loaded_model._layers.pop()
# loaded_model._layers.pop()
# loaded_model._layers.pop()
# loaded_model.summary()

In [163]:
# if tf.config.list_physical_devices('GPU'):
#     physical_devices = tf.config.list_physical_devices('GPU')
#     tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
#     tf.config.experimental.set_virtual_device_configuration(physical_devices[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])

# tf.config.gpu.set_per_process_memory_fraction(0.75)
# tf.config.gpu.set_per_process_memory_growth(True)


In [164]:
# predict_input = loaded_tokenizer.encode(test_val,
#                                  truncation=True,
#                                  padding=True,
#                                  return_tensors="tf")

# output = loaded_model(predict_input)[0]

# prediction_value = tf.argmax(output, axis=1).numpy()[0]

In [67]:
# type(depressed_dataset_1)
# depressed_dataset_1.head(10)

In [68]:
import pickle
import os
embed_path = r"D:\Workspace\OVGU\SM_Depression\scripts\embeddings_sentences"
os.chdir(embed_path)
@tf.autograph.experimental.do_not_convert
def get_store_embeddings(dataframe,dataframe_name):
#     sentences = []
#     row_count = dataframe.shape[0]
#     for i in range(0, row_count):
#         sentences.append(dataframe.iloc[i, 14])
    print(dataframe_name)
    BATCH_SIZE = 50
    print(len(dataframe.cleanData.tolist()))
#     embeddings = [loaded_tokenizer.encode(sent) for sent in sentences]
    tokenized_text = loaded_tokenizer.batch_encode_plus(dataframe.cleanData.tolist(),
                                           add_special_tokens = True,
                                           pad_to_max_length = True,
                                           max_length = 350,
                                            return_token_type_ids=True,
                                           return_tensors = 'tf')
    
    dataset = tf.data.Dataset.from_tensor_slices(tokenized_text['input_ids']).batch(BATCH_SIZE)
#     dataset = tf.data.Dataset.from_tensor_slices((tokenized_text['input_ids'],
#                                               tokenized_text['attention_mask'],
#                                               tokenized_text['token_type_ids'])).batch(3)

#     sample = next(iter(dataset))

    embeddings_array = np.empty([BATCH_SIZE,768])
    for batch in dataset:
        result = loaded_model(batch)  

#         print(type(result['hidden_states'][6]))
#         print(len(result['hidden_states'][6]))
        embeddings_batch = np.stack(list(result['hidden_states'][6][:,0,:]))
#         print(type(embeddings_batch))
#         print(embeddings_batch.shape)
        embeddings_array = np.concatenate((embeddings_array, embeddings_batch), axis = 0)


#     print(embeddings_array.shape)
#     print(embeddings_array)
    embeddings_array = np.delete(embeddings_array, np.s_[:BATCH_SIZE], 0)
    print(embeddings_array.shape)
#     print(embeddings_array)
#     embeddings_array = embeddings_array[:,0,:]
#     print(embeddings_array.shape)
    embed_name = dataframe_name +"_embedding.pkl"
    print(embed_name)
#     for sentence, embedding in zip(dataframe.cleanData.tolist(), embeddings_array):
#         print("Sentence:", sentence)
#         print("Embedding:", embedding)
#         print(embedding.shape)
#         print("")
    #Store sentences & embeddings on disc
    with open(embed_name, "wb") as fOut:
        pickle.dump(dict(zip(dataframe.cleanData.tolist(), embeddings_array)), fOut, protocol=pickle.HIGHEST_PROTOCOL)
#     return embeddings_array


In [82]:
# get_store_embeddings(depressed_dataset_1,"depressed_dataset_1")
# get_store_embeddings(depressed_dataset_2,"depressed_dataset_2")
# get_store_embeddings(depressed_dataset_3,"depressed_dataset_3")
# get_store_embeddings(depressed_dataset_4,"depressed_dataset_4")
# get_store_embeddings(depressed_dataset_5,"depressed_dataset_5")
# get_store_embeddings(depressed_dataset_6,"depressed_dataset_6")
# get_store_embeddings(depressed_dataset_7,"depressed_dataset_7")
# get_store_embeddings(depressed_dataset_8,"depressed_dataset_8")
# get_store_embeddings(depressed_dataset_9,"depressed_dataset_9")
# get_store_embeddings(depressed_dataset_10,"depressed_dataset_10")
# get_store_embeddings(depressed_dataset_11,"depressed_dataset_11")
# get_store_embeddings(depressed_dataset_12,"depressed_dataset_12")
# get_store_embeddings(depressed_dataset_13,"depressed_dataset_13")
# get_store_embeddings(depressed_dataset_14,"depressed_dataset_14")

depressed_dataset_14
887
(887, 768)
depressed_dataset_14_embedding.pkl


In [ ]:
#### RETurning function


# embeddings_array_1 = get_store_embeddings(depressed_dataset_1,"depressed_dataset_1")
# embeddings_array_2 = get_store_embeddings(depressed_dataset_2,"depressed_dataset_2")
# embeddings_array_3 = get_store_embeddings(depressed_dataset_3,"depressed_dataset_3")
# embeddings_array_4 = get_store_embeddings(depressed_dataset_4,"depressed_dataset_4")
# embeddings_array_5 = get_store_embeddings(depressed_dataset_5,"depressed_dataset_5")
# embeddings_array_6 = get_store_embeddings(depressed_dataset_6,"depressed_dataset_6")
# embeddings_array_7 = get_store_embeddings(depressed_dataset_7,"depressed_dataset_7")
# embeddings_array_8 = get_store_embeddings(depressed_dataset_8,"depressed_dataset_8")
# embeddings_array_9 = get_store_embeddings(depressed_dataset_9,"depressed_dataset_9")
# embeddings_array_10 = get_store_embeddings(depressed_dataset_10,"depressed_dataset_10")
# embeddings_array_11 = get_store_embeddings(depressed_dataset_11,"depressed_dataset_11")
# embeddings_array_12 = get_store_embeddings(depressed_dataset_12,"depressed_dataset_12")
# embeddings_array_13 = get_store_embeddings(depressed_dataset_13,"depressed_dataset_13")
# embeddings_array_14 = get_store_embeddings(depressed_dataset_14,"depressed_dataset_14")

In [105]:
#Sentences are encoded by calling model.encode()
# # embeddings = [loaded_tokenizer.encode(sent,truncation=True,padding=True,add_special_tokens = True) for sent in sentences]
# embeddings = loaded_tokenizer.batch_encode_plus(sentence,
#                                            add_special_tokens = True,
#                                             return_attention_masks = True,
#                                            pad_to_max_length = True,
#                                            max_length = 350,
#                                            return_tensors = 'tf')['input_ids']

# # print(sentence)
# # print(embeddings)
# #Print the embeddings
# for sentencea, embedding in zip(sentence, embeddings):
#     print("Sentence:", sentencea)
#     print("Embedding:", embedding)
#     print("")

In [106]:
# def get_store_embeddings(dataframe):
#     ### Change the dataframe.cleanData.values to your list containing data. and pass the list as arguement
#     ###        when calling the function. If you pass dataframe, mention the column whose values you need
#     ###        to encode like how i've mentioned here. 
#     embeddings = loaded_tokenizer.batch_encode_plus(dataframe.cleanData.values,
#                                            add_special_tokens = True,
#                                            pad_to_max_length = True,
#                                            max_length = 350,
#                                            return_tensors = 'tf')['input_ids']

#     return embeddings


# get_store_embeddings(depressed_dataset_1)